In [1]:
#Gathering Data manually from google

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#path to directory with images-- /content/drive/MyDrive/Python and DS/Minor Project/images

In [5]:
#preprocessing data

import os 
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize

target = []
images = []
flat_data = []

DATADIR = "/content/drive/MyDrive/Python and DS/Minor Project/images"
CATEGORIES = ["cat","dog"]
for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    #plt.imshow(img_array)
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    target.append(class_num)

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.4,random_state=42, stratify=target)

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm 
param_grid = [
           
          {'C':[1,10,100,1000],'kernel': ['linear']},
          {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']}
           
]

svc = svm.SVC(probability=True)
model = GridSearchCV(svc,param_grid)
model.fit(x_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [8]:
y_pred = model.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0])

In [9]:
y_test

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [10]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [11]:
accuracy_score(y_pred , y_test)

0.5555555555555556

In [12]:
confusion_matrix(y_pred,y_test)

array([[11,  9],
       [ 7,  9]])

In [13]:
import joblib 
joblib.dump(model,"img_model")

['img_model']

In [14]:
!pip install streamlit -- quiet 
!pip install pyngrok --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 6.1MB/s 
ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet
     |████████████████████████████████| 747kB 5.1MB/s 


In [15]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
import numpy as np
from PIL import Image
st.set_option("deprecation.showfileUploaderEncoding",False)
st.title("Cat and Dog classifier")
st.text("Upload the Image")

model = joblib.load("https://drive.google.com/file/d/1brh3wISUqakqd5lftlo7Dgqay7R9iQl8/view?usp=sharing")

uploaded_file = st.file_uploader("Choose an image..",type="jpg")
if uploaded_file is not None:
  img = Image.open(uploaded_file)
  st.image(img,caption = "Uploaded Image")

  if st.button("Predict"):
    CATEGORIES = ["cat","dog"]
    st.write("Result..")
    flat_data=[]
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    st.write(img.shape)
    y_out = model.predict(flat_data)
    y_out = CATEGORIES[y_out[0]]
    st.title(f"Predicted Output: {y_out}")
    q = model.predict_proba(flat_data)
    for index,item in enumerate(CATEGORIES):
      st.write(f"{item} : {q[0][index]*100}%")


Writing app.py


In [16]:
!nohup streamlit run app.py &
url = ngrok.connect(port="8501")
url

nohup: appending output to 'nohup.out'
nohup: failed to run command 'streamlit': No such file or directory


<NgrokTunnel: "http://e7ef4e7d3fad.ngrok.io" -> "http://localhost:80">

In [17]:
!pip install pipreqs
!pipreqs /content

INFO: Successfully saved requirements file in /content/requirements.txt
